<h1>Introduction to <i style="color:red;">SQLAlchemy</i></h1>

In [1]:
%%HTML
<img src="sqla_logo.png" width="300"/>

## <i>by Jack Camier, Python Developer</i>

SQLAlchemy is well documented at: https://www.sqlalchemy.org/

<h2>created by <i style="color:blue;">Mike Bayer</i></h2>

https://github.com/zzzeek

## There are a lot of tutorials on the subject, honestly too many to count.
## I hope to provide a tutorial that provides, some cookbook recipes (code snippets) and a little bit more visualization via a Jupyter Notebook to help developers better grasp the concepts than what I have found on the internet
<hr>

## "SQLAlchemy is the Python SQL toolkit and Object Relational Mapper that gives application developers the full power and flexibility of SQL" - Mike Bayer
https://www.sqlalchemy.org/
<hr>

## Our tutorial today will be using SQLite

In [2]:
%%HTML
<img src="sqlite_logo.gif" width="200"/>

https://www.sqlite.org/index.html

## The reason being you won't have to download or install anything in regards to a database perspective to do this tutorial 
## Also, `sqlite3` comes built-in with standard `Cpython`

In [3]:
import sqlite3 # That's it!

## "SQLite is a C-language library that implements a small, fast, self-contained, high-reliability, full-featured, SQL database engine. SQLite is the most used database engine in the world. SQLite is built into all mobile phones and most computers and comes bundled inside countless other applications that people use every day."
https://www.sqlite.org/index.html

## Now you could do things like the following to work with sqlite using the standard library `sqlite3` such as:

In [20]:
import uuid

sql_values = {
"id": str(uuid.uuid4()),
"first_name": "Bill",
"last_name": "Smith",
"email": "b.smith@mail.com"
}

sql_values

{'id': '85b7a2de-87bc-4c6d-b891-73f0e873fb22',
 'first_name': 'Bill',
 'last_name': 'Smith',
 'email': 'b.smith@mail.com'}

In [21]:
# Create connection session and database
conn = sqlite3.connect('example.db')

# Create cursor and assign it to `c`
c = conn.cursor()

# Create table using raw SQL
c.execute('''CREATE TABLE users (id text, first_name text, last_name text, email text)''')

In [22]:
# Insert a row of data 
c.execute('INSERT INTO users VALUES (:id, :first_name, :last_name, :email)', sql_values)
conn.commit()

In [23]:
# Select the data and print out row results
sql = c.execute('''SELECT * from users''')
for row in sql:
    print(row)

('85b7a2de-87bc-4c6d-b891-73f0e873fb22', 'Bill', 'Smith', 'b.smith@mail.com')


In [24]:
# Remove temporary `example.db` database
import os
os.remove("example.db")

## For simple things like this, we probably don't need SQLAlchemy. But normally within a short period of time, requirments of a project or feautures of a product change rapidly and we need to do something more like:

In [8]:
"""
select 
  'today - ' || to_char(trunc(sysdate),'Mon FMDDFM'),
  trunc(sysdate) as deadline
from dual 
UNION
select 
  'tomorrow - '|| to_char(trunc(sysdate+1),'Mon FMDDFM'), 
  trunc(sysdate+1) as deadline
from dual 
UNION
select
  'next week - '|| to_char(trunc(sysdate+7),'Mon FMDDFM'), 
  trunc(sysdate+7) as deadline
from dual 
UNION
select 
  'next month - '|| to_char(trunc(ADD_MONTHS(sysdate,1)),'Mon FMDDFM'), 
  trunc(ADD_MONTHS(sysdate,1)) as deadline 
from dual
UNION
select 
  name || ' - ' || to_char(deadline, 'Mon FMDDFM'), 
  deadline
from ticket_deadlines 
where project_id = :project_id
and deadline >= trunc(sysdate)
order by deadline
"""
# https://philip.greenspun.com/sql/complex-queries.html

"\nselect \n  'today - ' || to_char(trunc(sysdate),'Mon FMDDFM'),\n  trunc(sysdate) as deadline\nfrom dual \nUNION\nselect \n  'tomorrow - '|| to_char(trunc(sysdate+1),'Mon FMDDFM'), \n  trunc(sysdate+1) as deadline\nfrom dual \nUNION\nselect\n  'next week - '|| to_char(trunc(sysdate+7),'Mon FMDDFM'), \n  trunc(sysdate+7) as deadline\nfrom dual \nUNION\nselect \n  'next month - '|| to_char(trunc(ADD_MONTHS(sysdate,1)),'Mon FMDDFM'), \n  trunc(ADD_MONTHS(sysdate,1)) as deadline \nfrom dual\nUNION\nselect \n  name || ' - ' || to_char(deadline, 'Mon FMDDFM'), \n  deadline\nfrom ticket_deadlines \nwhere project_id = :project_id\nand deadline >= trunc(sysdate)\norder by deadline\n"

## "I wrote a reporting query once that was 700 lines long and visited 27 different tables in lookups or joins" - Greg Kemnitz
https://www.quora.com/Whats-the-most-complex-SQL-query-you-ever-wrote

## Figuring out how to do this in raw SQL can be very challenging if you are not a DBA or well versed in SQL syntax.
## With SQLAlchemy we can use the abstraction layer and write python code we are more familiar with and let the toolkit do the SQL magic behind the scenes

## First there are two main components of SQLAlchemy:
## "SQLAlchemy consists of two distinct components, known as the <i style="color:red;">Core</i> and the <i style="color:red;">ORM</i>. 
## The Core is itself a fully featured SQL abstraction toolkit, providing a smooth layer of abstraction over a wide variety of DBAPI implementations and behaviors, as well as a SQL Expression Language which allows expression of the SQL language via generative Python expressions."

## "The Object Relational Mapper (ORM) is then an optional package which builds upon the Core."
https://www.sqlalchemy.org/features.html

In [43]:
%%HTML
<img src="sqla_arch_small.png" width="500"/>

https://docs.sqlalchemy.org/en/13/intro.html

## To get sqlalchemy installed, all you have to do is:
`pip install SQLAlchemy`
## Then

In [10]:
import sqlalchemy

## In this talk, I am using version:

In [11]:
sqlalchemy.__version__

'1.3.9'

# To get started, we will commence looking at the Core component of SQLAlchemy

## Please note that individual database flavors, also called, `dialects` enumerate the available DBAPIs for each database
## Most seasoned developers learn that each database has its own nuances and slight variations of SQL syntax depending on the database. 
## Here are a list of popular `dialects`:

# - PostgreSQL
https://docs.sqlalchemy.org/en/13/dialects/postgresql.html
# - MySQL
https://docs.sqlalchemy.org/en/13/dialects/mysql.html
# - SQLite
https://docs.sqlalchemy.org/en/13/dialects/sqlite.html
# - Oracle
https://docs.sqlalchemy.org/en/13/dialects/oracle.html
# - Microsoft SQL Server
https://docs.sqlalchemy.org/en/13/dialects/mssql.html
<hr>

## Since our focus will be SQLite, here the main data type affinities:
* TEXT
* NUMERIC
* INTEGER
* REAL
* BLOB

https://www.sqlite.org/datatype3.html#affinity
<hr>

## "The SQLAlchemy Expression Language presents a system of representing relational database structures and expressions using Python constructs. These constructs are modeled to resemble those of the underlying database as closely as possible, while providing a modicum of abstraction of the various implementation differences between database backends."
https://docs.sqlalchemy.org/en/13/core/tutorial.html

##  <i style="color:red;">Huh!?</i> Translation:
## `SQLAlchemy Expression Language` is a way to represent everything dealing with a database using Python. 

# For this tutorial we will use an in-memory-only SQLite database. 

In [30]:
from sqlalchemy import create_engine

# To make the connection to the sqlite database
engine = create_engine('sqlite:///:memory:', echo=True)


## The `echo` flag is a shortcut to setting up SQLAlchemy logging using Python’s standard logging module.
## With it enabled, we’ll see all the generated SQL produced. Very helpful to see visually what is being done behind the scenes

## "The `engine` is the starting point for any SQLAlchemy application. It’s “home base” for the actual database and its DBAPI, delivered to the SQLAlchemy application through a connection pool and a Dialect..."
https://docs.sqlalchemy.org/en/13/core/engines.html

In [31]:
%%HTML
<img src="sqla_engine_arch.png" width="700"/>

## To use the `create_engine` method to an actual sqlite database, here are some examples:

In [37]:
# where <path> is relative:
engine = create_engine('sqlite:///foo.db')

# Linux/Mac - 4 initial slashes in total
engine = create_engine('sqlite:////absolute/path/to/foo.db')

# Windows
engine = create_engine('sqlite:///C:\\path\\to\\foo.db')

In [58]:
## I personally like to do something more like:

PROJECT_DIR = os.path.dirname(os.path.abspath(''))

# Note you can also do:

# os.path.dirname(os.path.abspath(__file__))

# But this only works in pure python .py files, not in Jupyter notebooks .ipynb

PROJECT_DIR

'/Users/jacquescamier/JupyterProjects'

## Note: the file path will be different for you since `os.path.dirname` etc. is specific to your computer

In [59]:
database_path = os.path.join(PROJECT_DIR, 'your_database.db')
sqlite_path = f"sqlite:///{database_path}"

sqlite_path

'sqlite:////Users/jacquescamier/JupyterProjects/your_database.db'

In [60]:
engine = create_engine(sqlite_path_str)

## Let's go back to our in-memory example

In [61]:
engine = create_engine('sqlite:///:memory:', echo=True)